In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [5]:
loader = PyPDFLoader("wardley-map-rag-app.pdf")

In [6]:
pages = loader.load_and_split()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 50)
chunks = text_splitter.split_documents(pages)

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [9]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)

In [10]:
retriever = vectorstore.as_retriever()

In [11]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [12]:
llm = ChatOpenAI()

In [13]:
template = """SYSTEM: You are a question answer bot. 
                 Be factual in your response.
                 Respond to the following question: {question} only from 
                 the below context :{context}. 
                 If you don't know the answer, just say that you don't know.
               """
prompt = PromptTemplate.from_template(template)

In [14]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
chain.invoke("What's the sick leave policy?")

"I'm sorry, I don't have enough information to provide an answer to the question about the sick leave policy."

In [16]:
chain.invoke("How do Wardley Maps help in business strategy?")

"Wardley Maps provide a visual representation of a company's strategy, helping to identify and understand the landscape and purpose of the business. They can be helpful in determining the direction and priorities for the company based on the context in which it operates."

In [17]:
chain.invoke("Why maps are important to win a war?")

'Maps are important to win a war because they provide topographical intelligence that can be decisive in battles throughout history. They help military personnel understand the terrain, positions of troops, and plan strategic movements effectively. By utilizing maps, commanders can make informed decisions, anticipate enemy movements, and gain an advantage in warfare.'